In [1]:
import numpy as np

In [ ]:
stds=np.load("stds.npy")
centers=np.load("centers.npy")

In [6]:
print([np.average(stds[:,0]), np.average(stds[:,1]), np.average(stds[:,2])])
print([np.average(centers[:,0]), np.average(centers[:,1]), np.average(centers[:,2])])

[2.4871936568499184, 2.497301162037346, 2.435337257486224]
[87.34482758620689, 89.24137931034483, 82.96551724137932]
